# Segmentation Exudatae, Red Dot（+MA） and Hemorrage from IDRiD dataset and DIARETDB1

Compiler (Adam)
Learning rate (0.00005)
Accuracy metric: Dice coefficient [9] to be maximized
Loss metric: negative Dice coefficient (to be minimized)

In [ ]:
from matplotlib import pyplot as plt
from tensorflow import keras

import tensorflow as tf
import pandas as pd
import numpy as np
import random
import math
import keras.backend as K

import cv2
import os

WIDTH, HEIGHT = 1024, 1024
# NUM_CLASSES = 3 # 0. Background, 1, Exudatas(Hard + Soft), 2. Red Dot + Hemorrhages + Microaneurysms
BATCH_SIZE = 1


EPOCHS = 10# 50

# On Colab
# MODEL_DIR = "./Diabetic Retinopathy/"
# BASE_DIR = 'F:\hcc\A. Segmentation\A. Segmentation/'

# On windows Os
MODEL_DIR = "./Diabetic Retinopathy/"# "C:/Users/yhu04/Diabetic Retinopathy/"
BASE_DIR = "./DR_data/"
psi = [1]*5
NUM_CLASSES = 3# 3
category_types = ["Background", "EX", "RHM"]
background = [255,255,255]
ex = [128,128,0]
rhm = [255,0,0]
label_colours = np.array([background,ex,rhm])
def visualize(temp, plot=True):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,3):
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb

In [ ]:
def get_backbone(width = WIDTH, height = HEIGHT):
    backbone = keras.applications.VGG16(include_top=False, input_shape=(width, height, 3), weights = "imagenet")
    output_0, output_1, output_2, output_3, output_4, fin_output = [
        backbone.get_layer(layer_name).output
        for layer_name in ["block1_conv2","block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3", "block5_pool"]
    ]
   
        
    return keras.Model(
        inputs=[backbone.inputs],
        outputs=[ fin_output, output_4, output_3, output_2, output_1, output_0 ],
        )

In [ ]:
def twice_Conv2D(input_image, num_filters, i):
    for j in range(2):
        output = keras.layers.Conv2D(num_filters, 3, 1, 
                                    padding = "same",
                                    kernel_initializer='he_normal',
                                    name=f"ccblock{i+1}_Conv{j+1}")(input_image)
        
        output = keras.layers.BatchNormalization(name=f"ccblock{i+1}_BN{j+1}")(output)
        output = keras.layers.ReLU(name = f"ccblock{i+1}_ReLU{j+1}")(output)
#         print(output.shape)
        
    return output

    
    
def Unet(psi = psi, num_classes = NUM_CLASSES, width = WIDTH, height = HEIGHT): 
    
    input_image = keras.Input(shape=(width, height, 3), name="ccinput")

    output_list = get_backbone()(input_image, training = False)  
    output = output_list[0]# 即fin_output
    
    outC = keras.layers.MaxPool2D(32,32)(output)
    outC = keras.layers.Flatten(name='ccflatten')(outC)
    outC = keras.layers.Dense(units=32)(outC)
    outC = keras.layers.ReLU()(outC)
    outC = keras.layers.Dense(units=int(np.sum(psi)),activation="softmax",name=f"grad")(outC)
#     print(outC.shape)# (None,5)?or 3
    
    for i in range(2):
        output = keras.layers.Conv2D(512, 3, 1, 
                                     padding = "same",
                                     kernel_initializer='he_normal',
                                     name=f"ccblock{0}_Conv{i}")(output)
        output = keras.layers.BatchNormalization(name=f"ccblock{0}_BN{i}")(output)
        output = keras.layers.ReLU(name=f"ccblock{0}_ReLU{i}")(output)
          
    for i, filters in enumerate([256, 128, 64, 32, 16]):
        output = keras.layers.UpSampling2D(2, name=f"ccblock{i+1}_UpSampling{0}")(output)

        output = keras.layers.Conv2D(output.shape[-1], 2, 1, 
                                     padding = "same",
                                     kernel_initializer='he_normal',
                                     name=f"ccblock{i+1}_Conv{0}")(output)

        output = keras.layers.BatchNormalization(name=f"ccblock{i+1}_BN{0}")(output)
        output = keras.layers.ReLU(name=f"ccblock{i+1}_ReLU{0}")(output)
        output = keras.layers.concatenate([output, output_list[i+1]], name = f"ccblock{i+1}_concat")
        output = twice_Conv2D(output, filters, i)

                
    output = keras.layers.Conv2D(num_classes, 1, 1, 
                                 padding = "same", 
                                 activation = "softmax", 
                                 kernel_initializer=tf.initializers.RandomNormal(0.0, 0.01),
                                 bias_initializer = tf.constant_initializer(-np.log((1 - 0.01) / 0.01)),
                                 name = f"grad{5}_Conv_ReLU")(output)
    
    # (None, 1024, 1024, 3)
    
    model = keras.models.Model(inputs = input_image, outputs=[output,outC])
    return model
# Unet(3,1024,1024)

In [ ]:
def mask_softmax(x,mask):
    logits = tf.exp(x) * mask / tf.reduce_sum(tf.exp(x) * mask)
#     print(logits)
    return logits

In [ ]:
# train_label = [0,1]
img = tf.keras.Input(shape=(1024, 1024, 3), name="ccinput")
grad = [2]
def LabelGenerator(x,y,class_nb = psi,num_classes = NUM_CLASSES,batch_size = BATCH_SIZE):
   
    filters = [64,128,256,512,512]
#     x = np.reshape(x, ((batch_size,) + x.shape))
#     print("encoder:",input_image.shape)# encoder: (None, 1024, 1024, 3)
    output_list = get_backbone()(x, training = False)  
    output = output_list[0]# 即fin_output
    
    outC = keras.layers.MaxPool2D(32,32)(output)
    outC = keras.layers.Flatten(name='ccflatten')(outC)
    outC = keras.layers.Dense(units=32)(outC)
    outC = keras.layers.ReLU()(outC)
    outC = keras.layers.Dense(units=int(np.sum(psi)),activation="softmax",name=f"classfc")(outC)
    
    
    index = tf.zeros([len(class_nb),np.sum(class_nb)]) + 1e-8
    index = index.numpy()
    
    for i in range(len(class_nb)):
        index[i, int(np.sum(class_nb[:i])):np.sum(class_nb[:i+1])] = 1
    
    index = tf.convert_to_tensor(index)
#     print(index)
#     print(y)# [3]
    mask = index[y]
#     print(mask)# tf.Tensor([1.e-08 1.e-08 1.e-08 1.e+00 1.e-08], shape=(5,), dtype=float32) grad=[3]
    
    
    classifier = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(filters[-1]),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dense(filters[-1]),
        
    ])
    
#     predict.shape (1, 512)
#     mask.shape (5,)
#     print("outC.shape",outC.shape)
    label_pred = mask_softmax(outC, mask)
#     print(label_pred.shape)# (None, 5)

    return label_pred  
LabelGenerator(img,grad)

In [ ]:
def Dice(y_true, y_pred, solo = True, num_classes = NUM_CLASSES):
   
    smooth = 0.0001# 0.0001
    if solo:
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                            depth=num_classes,
                            dtype=tf.float32,
                            )
    
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    
    return (numerator + smooth) / (denominator + smooth)

In [ ]:
def hist_info(n_cl, pred, gt):
    assert (pred.shape == gt.shape)  
    k = (gt >= 0) & (gt < n_cl)  
    labeled = np.sum(k)  
    correct = np.sum((pred[k] == gt[k]))  

    return np.bincount(n_cl * gt[k].astype(int) + pred[k].astype(int),
                       minlength=n_cl ** 2).reshape(n_cl, n_cl), labeled, correct

In [ ]:
def Dice_class(y_true, y_pred, solo = True, num_classes = 5):
    smooth = 0.0001# 0.0001
    if solo:
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                            depth=num_classes,
                            dtype=tf.float32,
                            )
        
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return (numerator + smooth) / (denominator + smooth)

In [ ]:
def Classify_ac(y_true, y_pred,solo = True, num_classes = 5):
    if solo:
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                            depth=num_classes,
                            dtype=tf.float32,
                            )
    correct_predict = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, 'float'))
    return accuracy

In [ ]:
class CategoricalFocalLoss(tf.losses.Loss):
    """
    公式 : loss = - y_true * alpha * ((1 - y_pred)^gamma) * log(y_pred)
        
    alpha: the same as weighting factor in balanced cross entropy, default 0.25
    gamma: focusing parameter for modulating factor (1-p), default 2.0

    y_true =  [[0., 1.0, 0.], [0., 0., 1.], [0., 1., 0.]]
    y_pred = [[0.70, 0.15, 0.15], [0.1, 0.8, 0.1], [0.25, 0.65, 0.1]]
    y_true = tf.cast(y_true, dtype= "float32")
    y_pred = tf.cast(y_pred, dtype= "float32")
    gamma=3.0
    alpha=0.25
    """
    def __init__(self, solo = True, num_classes = NUM_CLASSES, gamma = 2.0, alpha=0.25):
        super(CategoricalFocalLoss, self).__init__(reduction = 'auto', name = "CategoricalFocalLoss")
        self._num_classes = num_classes
        self._gamma = gamma
        self._alpha = alpha
        self._epsilon = 1e-07
        self.solo = solo
        
        

    def call(self, y_true, y_pred):
      
        if self.solo:
            y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                                depth=self._num_classes,
                                dtype=tf.float32,
                                )
         
        y_pred = tf.clip_by_value(y_pred, self._epsilon, 1.0 - self._epsilon)
        loss = - y_true * self._alpha * tf.math.pow((1 - y_pred), self._gamma) * tf.math.log(y_pred)
        """
        Another Code
        alpha = tf.where(tf.equal(y_true, 1.), alpha, (1.0 - self._alpha))
        pt = tf.where(tf.equal(y_true, 1.), y_pred, 1-y_pred)
        y_pred = tf.add(y_pred, self._epsilon)
        loss = alpha * tf.pow(1.0 - pt, self._gamma) * tf.multiply(y_true, -tf.math.log(y_pred))
        """
#      
        return tf.reduce_mean(loss)

In [ ]:
class CategoricalFocalLoss2(tf.losses.Loss):
   
    def __init__(self, solo = True, num_classes = 5, gamma = 2.0, alpha=0.25):
        super(CategoricalFocalLoss2, self).__init__( name = "CategoricalFocalLoss2")# reduction = 'auto',
        self._num_classes = num_classes
        self._gamma = gamma
        self._alpha = alpha
        self._epsilon = 1e-07
        self.solo = solo
        
        

    def call(self, y_true, y_pred):
      
        if self.solo:
            y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                                depth=self._num_classes,
                                dtype=tf.float32,
                                )
         
        y_pred = tf.clip_by_value(y_pred, self._epsilon, 1.0 - self._epsilon)
        loss = - y_true * self._alpha * tf.math.pow((1 - y_pred), self._gamma) * tf.math.log(y_pred)
       
        return tf.reduce_mean(loss)

In [ ]:
def generalized_dice_coef_fun(smooth=0, solo=True):
    def generalized_dice(y_true, y_pred):
        # Compute weights: "the contribution of each label is corrected by the inverse of its volume"
        if solo:
            y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                                depth=NUM_CLASSES,
                                dtype=tf.float32,
                                )
        w = K.sum(y_true, axis=(0, 1, 2))
        w = 1 / (w ** 2 + 0.00001)
      
        # Compute gen dice coef:
        numerator = y_true * y_pred
        numerator = w * K.sum(numerator, axis=(0, 1, 2))
        numerator = K.sum(numerator)
 
        denominator = y_true + y_pred
        denominator = w * K.sum(denominator, axis=(0, 1, 2))
        denominator = K.sum(denominator)
 
        gen_dice_coef = numerator / denominator
 
        return  2 * gen_dice_coef
    return generalized_dice
 
def generalized_dice_loss_fun(smooth=0):
    def generalized_dice_loss(y_true,y_pred):
        return 1 - generalized_dice_coef_fun(smooth=smooth)(y_true=y_true,y_pred=y_pred)
    return generalized_dice_loss

In [ ]:
activation = lambda y: y
def cont_kappa(y_true, y_pred, activation=None):
    ''' continuos version of quadratic weighted kappa '''
    y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                                depth=self.num_class,
                                dtype=tf.float32,
                                )
    n = len(y_pred)
    y = tf.cast(tf.expand_dims(y_pred,0),dtype=tf.int32)
    pred = tf.cast(tf.expand_dims(tf.squeeze(y_true,-1),0),dtype=tf.int32)# y_true.squeeze(-1).unsqueeze(0)
    if activation is not None:
        pred = activation(pred)
    wo = (pred - y)**2
    we = (pred - tf.transpose(y))**2
    
    return 1 - (tf.cast(n,dtype=tf.int32) * tf.reduce_sum(wo) / tf.reduce_sum(we))

kappa_loss = lambda pred, y: 1 - cont_kappa(pred, y)  # from 0 to 2 instead of 1 to -1
class_weights = [0.09759672877367588, 4.099062608494387, 0.08911005670639972, 0.3726420553176716, 0.34158855070786565]
class_weights = tf.convert_to_tensor(class_weights,dtype=tf.float32)
# class_weights = np.array([0, 4, 0, 0, 0])
class grad_Loss(tf.losses.Loss):   
    def __init__(self,num_class=5, alpha=class_weights, gamma=2.0, second_loss=kappa_loss, second_mult=0.5):
        super(grad_Loss, self).__init__()
        
        self.alpha = tf.constant(alpha,dtype=tf.float32)# dtype=tf.float32
        self.gamma = gamma
        
        self.num_class = num_class
        
        self.second_loss = second_loss
        
        self.second_mult = second_mult
    def call(self,y_true,y_pred):
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32),
                                depth=self.num_class,
                                dtype=tf.float32,
                                )
   
        alpha_choice = tf.gather(self.alpha,y_true)
       
        CE_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred)
        pt = tf.exp(-CE_loss)
        
        yy = tf.argmax(y_pred,0)
        
        F_loss = alpha_choice*(1-pt)**self.gamma*CE_loss
        F_loss = tf.reduce_mean(F_loss)
        
        loss = F_loss + self.second_mult*self.second_loss(y_true,y_pred)
        return loss

In [ ]:
def dense_to_one_hot(labels_dense, num_classes):
    """
    :param labels_dense:
    :param num_classes:
    label number must start from zero,such as:0,1,2,3,4,...
    :return:
    """
    labels_dense = np.array(labels_dense)
    num_labels = labels_dense.shape[0]
    labels_one_hot = np.zeros((num_labels, num_classes))
    # method one
    for index in range(num_labels):
        for classindex in range(num_classes):
            if labels_dense[index] == classindex:
                labels_one_hot[index, classindex] = 1
   
    return labels_one_hot

In [ ]:
from sklearn.utils import shuffle
class Dataset_Generator():
    def __init__(self,
                 base_dir = BASE_DIR,
                 num_classes = NUM_CLASSES,
                 batch_size = BATCH_SIZE,
                 height = HEIGHT,
                 width = WIDTH,
                 epochs = EPOCHS,
                ):
        
        self.base_dir = BASE_DIR
        self.num_classes = float(num_classes)
        self.batch_size = batch_size
        self.height = HEIGHT
        self.width = WIDTH
        self.epochs = epochs
        #self.images_list = []
        self.images_list = os.listdir(self.base_dir + "Training/images/")
        random.shuffle(self.images_list)
        
    def __del__(self):
        print("Dataset Generator is destructed")
            
    def _preprocessor(self):
        
        try:
            os.mkdir(self.base_dir+"Training")
            os.mkdir(self.base_dir+"Test")
            os.mkdir(self.base_dir+"Training/images")
            os.mkdir(self.base_dir+"Test/images")
            os.mkdir(self.base_dir+"Training/masks")
            os.mkdir(self.base_dir+"Test/masks")
        except FileExistsError:
            pass
        
        idrid_cnt = diaretdb_cnt = 0 
       
        image_list = os.listdir(self.base_dir + "image/")
        for i, file_name in enumerate(image_list):
            image_list[i] = file_name.split(".")[0]
        image_list.sort()

        mask_class_dir = ["MA", "HE", "EX", "SE"]
        mask_file_list = []

        for cls in mask_class_dir:
            mask_file_list.append(os.listdir(self.base_dir + f"mask/{cls}"))

        zero_1 = np.zeros([2848, 4288], dtype = np.uint8)
        zero_2 = np.zeros([1152, 1500], dtype = np.uint8)

        loss_cnt = 0
        
        for i, file_name in enumerate(image_list):
            if "IDRiD" in file_name:
                zero = zero_1
                thres = 1
            elif "image" in file_name:
                zero = zero_2
                # [63, 127, 189, 252]
                thres = 127

            mask_list = []
            # 1. mask
            for cls in range(4):
                
                if "IDRiD" in file_name:
                    mask_file_name = f"{file_name}_{mask_class_dir[cls]}.tif"
                elif "image" in file_name:
                    mask_file_name = f"{file_name}.png"

                if mask_file_name in mask_file_list[cls]:
                    mask = cv2.imread(f"{self.base_dir}mask/{mask_class_dir[cls]}/{mask_file_name}", 0)
                    _, mask = cv2.threshold(mask, thres, 1, cv2.THRESH_BINARY)
                else:
                    mask = zero
                mask_list.append(mask)

            # 2. mask fusion
            Class_1 = cv2.bitwise_or(mask_list[0], mask_list[1]) * 100
            Class_2 = cv2.bitwise_or(mask_list[2], mask_list[3]) * 200
            mask = Class_1 + Class_2
            del Class_1, Class_2, mask_list

            if np.all(mask == zero):
                loss_cnt += 1
                print(f"{file_name} has no mask")
            else:
                # Binaryzation
                if "IDRiD" in file_name:
                    file_name = f"{file_name}.jpg"
                elif "image" in file_name:
                    file_name = f"{file_name}.png"
                img = cv2.imread(f"{self.base_dir}image/{file_name}")

                if "IDRiD" in file_name:
                    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    if i==3 or i == 10:
                        thres = 10
                    else:
                        thres = 30

                    _, binary_img = cv2.threshold(gray_img, thres, 255, cv2.THRESH_BINARY)
                    del gray_img

                    contours, hierachy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

                    x_min = np.min(contours[-1], axis = 0)
                    x_max = np.max(contours[-1], axis = 0)
                    x_min, x_max = x_min[0][0], x_max[0][0]
                    del contours, hierachy

                    img = img[:, x_min:x_max+1]
                    mask = mask[:, x_min:x_max+1]

                    if (x_max-x_min)/2848 >= 1.25:
                        pad_left, pad_right = 0, 0
                    else:
                        pad_left, pad_right = 200, 200

                    img = cv2.copyMakeBorder(img, 500, 500, pad_left, pad_right, cv2.BORDER_CONSTANT,value=0)
                    mask = cv2.copyMakeBorder(mask, 500, 500, pad_left, pad_right, cv2.BORDER_CONSTANT,value=0)

                elif "image" in file_name:
                    img = cv2.copyMakeBorder(img, 174, 174, 0, 0, cv2.BORDER_CONSTANT,value=0)
                    mask = cv2.copyMakeBorder(mask, 174, 174, 0, 0, cv2.BORDER_CONSTANT,value=0)

                img = cv2.resize(img, dsize=(self.height, self.width), interpolation=cv2.INTER_AREA)
                mask = cv2.resize(mask, dsize=(self.height, self.width), interpolation=cv2.INTER_AREA)

                if "IDRiD" in file_name and idrid_cnt < 60:
                    cv2.imwrite(f'{self.base_dir}Training/images/{file_name}', img)
                    cv2.imwrite(f'{self.base_dir}Training/masks/{file_name}', mask)
#                     cv2.imwrite(f'{self.base_dir}Training/grad_label/{file_name}', grad)# cc
                    idrid_cnt += 1

                elif "IDRiD" in file_name and idrid_cnt >= 60:
                    cv2.imwrite(f'{self.base_dir}Test/images/{file_name}', img)
                    cv2.imwrite(f'{self.base_dir}Test/masks/{file_name}', mask)
#                     cv2.imwrite(f'{self.base_dir}Training/grad_label/{file_name}', grad)# cc
                    idrid_cnt += 1

                elif "image" in file_name and diaretdb_cnt < 40:
                    cv2.imwrite(f'{self.base_dir}Training/images/{file_name}', img)
                    cv2.imwrite(f'{self.base_dir}Training/masks/{file_name}', mask)
#                     cv2.imwrite(f'{self.base_dir}Training/grad_label/{file_name}', grad)# cc
                    diaretdb_cnt += 1

                elif "image" in file_name and diaretdb_cnt >= 40:
                    cv2.imwrite(f'{self.base_dir}Test/images/{file_name}', img)
                    cv2.imwrite(f'{self.base_dir}Test/masks/{file_name}', mask)
#                     cv2.imwrite(f'{self.base_dir}Training/grad_label/{file_name}', grad)# cc
                    diaretdb_cnt += 1

                print(f"{file_name} completed!")
        self.images_list = os.listdir(self.base_dir + "Training/images/")
        random.shuffle(self.images_list)
        print(f"Preprocessing completed!. Number of no mask data : {loss_cnt}")
        
    
    def _Image_Reshape(self, image, mask, grad):     
        
#         print(image.shape)# (1024,1024,3)
        
        image = np.reshape(image, ((self.batch_size,) + image.shape))

        
        mask = np.reshape(mask, ((self.batch_size,) + mask.shape))
        
#         grad = np.reshape(grad,((self.batch_size,) + np.array(grad).shape))
        
        grad = LabelGenerator(image,grad)

#         print("np.array(mask/200.",np.array(mask/200.))# np.array(mask/200. (1, 1024, 1024)#change mask to 100 will add lesions
#         print("np.array(image/255.",np.array(image/255))# np.array(image/255. (1, 1024, 1024, 3)
#         print(grad.shape)
        return ({'ccinput':np.array(image/255.)}, {'grad5_Conv_ReLU':np.array(mask/200),'grad':grad})# cc
    
    def train_generator(self, k):
        """
        Training Data Augmentation
        """
      
        if self.images_list:
            pass
        else:
            self._preprocessor()
        x_center, y_center = self.width/2, self.height/2
       
        
        for _ in range(self.epochs):
            
            # cc add grad_labels
            grad_train = pd.read_csv(self.base_dir+'Training/grad_label/id_dia.csv')
            x = grad_train['id_code']
            y = grad_train['diagnosis']
            x, y = shuffle(x, y)
            # cc get class stats
            n_classes = int(y.max()+1)
            # cc print(n_classes)# 
            class_weights = len(y) / grad_train.groupby('diagnosis').size().values.ravel()  # we can use this to balance our loss function
            class_weights *= n_classes / class_weights.sum()
#             print('class_weights:', class_weights.tolist())
            
            for i, file_name in enumerate(self.images_list):
                grad = []
                if 20*k-20 <= i < 20*k:
                    pass
                else:
             
                    img = cv2.imread(f"{self.base_dir}Training/images/{file_name}")
                   

                    mask = cv2.imread(f"{self.base_dir}Training/masks/{file_name}", 0)
                          
                    for i in range(len(x.values)):# cc
                        if(file_name == x.values[i]):
                            grad.append(y.values[i])
                    
                    yield self._Image_Reshape(img, mask, grad)
                    
                    flip_img = cv2.flip(img, 1)
                    
                    flip_mask = cv2.flip(mask, 1)
                    yield self._Image_Reshape(flip_img, flip_mask, grad)

                    for degree in range(90, 360, 90):
                        matrix = cv2.getRotationMatrix2D((x_center, y_center), degree, 1)

                        rot_img = cv2.warpAffine(img, matrix, (self.width, self.height))
                    
                        rot_mask = cv2.warpAffine(mask, matrix, (self.width, self.height))
                        yield self._Image_Reshape(rot_img, rot_mask, grad)

                        rot_flip_img = cv2.warpAffine(flip_img, matrix, (self.width, self.height))
                    
                        rot_flip_mask = cv2.warpAffine(flip_mask, matrix, (self.width, self.height))
                        yield self._Image_Reshape(rot_flip_img, rot_flip_mask, grad)


    def valid_generator(self, k):
        """
        Validataion Data Augmentation
        """
        x_center, y_center = self.width/2, self.height/2
        
        
        for _ in range(self.epochs):

            grad_train = pd.read_csv(self.base_dir+'Training/grad_label/id_dia.csv')
            x = grad_train['id_code']
            y = grad_train['diagnosis']
            x, y = shuffle(x, y)
            # cc get class stats
            n_classes = int(y.max()+1)
            # cc print(n_classes)# 
            class_weights = len(y) / grad_train.groupby('diagnosis').size().values.ravel()  # we can use this to balance our loss function
            class_weights *= n_classes / class_weights.sum()
#             print('class_weights:', class_weights.tolist())
#             class_weights: [0.09759672877367588, 4.099062608494387, 0.08911005670639972, 0.3726420553176716, 0.34158855070786565]
            
            for i, file_name in enumerate(self.images_list):
                grad = []
                if (20*k-20) <= i < 20*k:

                    img = cv2.imread(f"{self.base_dir}Training/images/{file_name}")
                   
                    mask = cv2.imread(f"{self.base_dir}Training/masks/{file_name}", 0)

                    for i in range(len(x.values)):# cc
                        if(file_name == x.values[i]):
                            grad.append(y.values[i])
                            
                    yield self._Image_Reshape(img, mask, grad)
                 
                   
                    flip_img = cv2.flip(img, 1)
                   
                    flip_mask = cv2.flip(mask, 1)
                    yield self._Image_Reshape(flip_img, flip_mask,grad)

                    for degree in range(90, 360, 90):
                        matrix = cv2.getRotationMatrix2D((x_center, y_center), degree, 1)
                            
                        
                        rot_img = cv2.warpAffine(img, matrix, (self.width, self.height))
                        
                        rot_mask = cv2.warpAffine(mask, matrix, (self.width, self.height))
                        yield self._Image_Reshape(rot_img, rot_mask, grad)

                        
                        rot_flip_img = cv2.warpAffine(flip_img, matrix, (self.width, self.height))
                       
                        rot_flip_mask = cv2.warpAffine(flip_mask, matrix, (self.width, self.height))
                        yield self._Image_Reshape(rot_flip_img, rot_flip_mask, grad)


                
    def test_generator(self):
        images_list = os.listdir(self.base_dir + "Test/images/")

        grad_test = pd.read_csv(self.base_dir+'Test/grad_label/id_dia.csv')
        x = grad_test['id_code']
        y = grad_test['diagnosis']
        x, y = shuffle(x, y)
        # cc get class stats
        n_classes = int(y.max()+1)
        # cc print(n_classes)# 
        class_weights = len(y) / grad_test.groupby('diagnosis').size().values.ravel()  # we can use this to balance our loss function
        class_weights *= n_classes / class_weights.sum()
#         print('class_weights:', class_weights.tolist())
            
        for i, file_name in enumerate(images_list):
            grad = []
           
            img = cv2.imread(f"{self.base_dir}Test/images/{file_name}")
        
            mask = cv2.imread(f"{self.base_dir}Test/masks/{file_name}", 0)
            
#             cv2.namedWindow("mask")     
#             cv2.imshow("mask", mask)    
#             cv2.waitKey()               
#             cv2.destroyAllWindows()               
            
            for i in range(len(x.values)):# cc
                
                if(file_name == x.values[i]):
                    grad.append(y.values[i])           
                            
            yield self._Image_Reshape(img, mask,grad)
dataset = Dataset_Generator()

gen = dataset.test_generator()

result = next(gen)

In [ ]:
## Training dataset check
dataset = Dataset_Generator()

gen = dataset.train_generator(1)
for i in range(720 * 4):
    result = next(gen)
    
    img = 'input'
    mask = 'mask'
    
#     grad = result[3][0]# 
#     print(grad)
#     img=np.expand_dims(img,axis=0)
   
#     print(img.shape)

    fig = plt.figure(i, figsize = (10,10))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(img)
    ax1.set_title('Image')
    ax1.axis("off")

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.imshow(mask, cmap = "bone")
    ax2.set_title('Ground Truth Mask')
    ax2.axis("off")
    if i == 3:
        break

In [ ]:
## Validation dataset check
gen = dataset.valid_generator(1)
for i in range(720):
    result = next(gen)
    img, mask = result[0][0], result[1][0]
    
#     grad = result[2][0]
#     print(grad)
    
    fig = plt.figure(i, figsize = (10,10))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(img)
    ax1.set_title('Image')
    ax1.axis("off")

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.imshow(mask, cmap = "bone")
    ax2.set_title('Ground Truth Mask')
    ax2.axis("off")
    if i == 3:
        break

In [ ]:
## Test dataset
test_dataset = tf.data.Dataset.from_generator(
                    dataset.test_generator,
                    (tf.float32, tf.int32),
                    (tf.TensorShape([1, HEIGHT, WIDTH, 3]), tf.TensorShape([1,  HEIGHT, WIDTH])),
                    )

del dataset

## Check Test set test
for i, element in enumerate(test_dataset):
    img = element[0][0].numpy()
    mask = element[1][0].numpy()
    fig = plt.figure(i, figsize = (10,10))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(img)
    ax1.set_title('Image')
    ax1.axis("off")

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.imshow(mask, cmap = "bone")
    ax2.set_title('Ground Truth Mask')
    ax2.axis("off")
    
    if i == 3:
        break
    plt.show()

del gen, test_dataset

In [ ]:
input_image = tf.keras.Input(shape=(1024, 1024, 3), name="ccinput")
dataset = Dataset_Generator()
model = Unet()
test_dataset = tf.data.Dataset.from_generator(
                dataset.test_generator,
                ({'ccinput': tf.float32},{ 'grad5_Conv_ReLU':tf.int32,  'grad':tf.int32}),# cc
                ({'ccinput':tf.TensorShape([1, HEIGHT, WIDTH, 3])}, { 'grad5_Conv_ReLU':tf.TensorShape([1,  HEIGHT, WIDTH]), 'grad':tf.TensorShape([1,5])}),
                )
predictions,classifications = model(input_image, training = True)
# print(classifications)# shape=(None, 5)
inference_model = tf.keras.Model(inputs=input_image, outputs=[predictions,classifications])# cc

for i,test in enumerate(test_dataset):
    img,mask_grad= test  
    prediction,classify = inference_model.predict(img)
    
    
    img = img["ccinput"][0].numpy()
    mask = mask_grad['grad5_Conv_ReLU'][0].numpy()
    
    prediction = prediction[0]
    prediction = tf.math.argmax(prediction, -1)
    prediction = prediction.numpy()

    fig = plt.figure(10, figsize = (20,20))
    ax1 = fig.add_subplot(1, 3, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax1.imshow(img)
    ax1.set_title('Image')
    ax1.axis("off")

    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(mask)
    ax2.set_title('Ground Truth Mask')
    ax2.axis("off")

    ax3 = fig.add_subplot(1, 3, 3)
    ax3.imshow(prediction)
    ax3.set_title('Prediction')
    ax3.axis("off")
    plt.show()

In [ ]:
class MODEL():
    def __init__(self,
                 model_dir = MODEL_DIR,
                 batch_size = BATCH_SIZE,
                 width = WIDTH,
                 height = HEIGHT,
                 k = 0,
                ):
        
        self.batch_size = batch_size
        self.loss_fn = generalized_dice_loss_fun()#CategoricalFocalLoss()
         
        self.loss_fc = CategoricalFocalLoss2()#grad_Loss()
        
        self.optimizer =  tf.keras.optimizers.Adam(learning_rate=0.00005)# 0.00005
        self.generator = Dataset_Generator()
        self.model_dir = MODEL_DIR
        self.optimal_k = k
        

        self.test_dataset = tf.data.Dataset.from_generator(
                        self.generator.test_generator,
                        ({'ccinput': tf.float32},{ 'grad5_Conv_ReLU':tf.int32,  'grad':tf.int32}),# cc tf.int32
                        ({'ccinput':tf.TensorShape([1, HEIGHT, WIDTH, 3])}, { 'grad5_Conv_ReLU':tf.TensorShape([1,  HEIGHT, WIDTH]), 'grad':tf.TensorShape([1,5])}),
                        )
        self.width = width
        self.height = height
    
    def __del__(self):
        print("MODEL is destructed")
        

    def Run_training(self, epochs=EPOCHS):        
        print("Model Complie....")        
        
        # K-fold: k = 5
        K = 5
        mean_Dice = mean_IoU = 0
        DiceIoU_list = []
        for k in range(1, K+1):
            model = Unet()
            model.compile(loss = {'grad5_Conv_ReLU':self.loss_fn,'grad':self.loss_fc} ,#'categorical_crossentropy'   
                          loss_weights = [1,0.8],
                          optimizer = self.optimizer,
                          metrics = {'grad5_Conv_ReLU':Dice,"grad":Dice_class}#Dice_class\Classify_ac Jaccard sparse_categorical_accuracy; acc;
#                           metrics = ["accuracy"]
                          )
            callbacks_list = [tf.keras.callbacks.ModelCheckpoint(
                                        filepath=os.path.join(
                                            f"{self.model_dir}U-Net_{k}.h5"),
                                        monitor="val_loss",
#                                         monitor="val_block5_Conv_ReLU_Dice",
                                        mode = "min",
                                        save_best_only=True,
                                        save_weights_only=False,
                                        verbose=1,
                                        ),
                              tf.keras.callbacks.EarlyStopping(
                                        monitor = 'val_loss',
#                                         monitor="val_block5_Conv_ReLU_Dice",
                                        mode = "min",
                                        min_delta = 0.01,
                                        patience = 5,
                                        )
                              ]
            print(f"{k}th fold Start Training....")
            
      
        
            history = model.fit(self.generator.train_generator(k),
                                steps_per_epoch = (K-1) * 20 * 8,
                                validation_data = self.generator.valid_generator(k),
                                validation_steps = 20 * 8,
                                callbacks = callbacks_list,
                                epochs = epochs,
                                batch_size = self.batch_size,
#                                 shuffle = True,
                                )
            
            loss = history.history['loss']
            val_loss = history.history['val_loss']
            dice = history.history["grad5_Conv_ReLU_Dice"]
            val_dice = history.history["val_grad5_Conv_ReLU_Dice"]
            iou = history.history["grad_Dice_class"]
            val_iou = history.history["val_grad_Dice_class"]
            
            DiceIoU_list.append( val_dice[-1] + val_iou[-1] )
            mean_Dice += val_dice[-1]
            mean_IoU += val_iou[-1]

            epochs_range = range(len(loss))
            
            plt.figure(k, figsize=(15, 5))

            plt.subplot(1, 3, 1)
            plt.plot(epochs_range, loss, label='Training Loss')
            plt.plot(epochs_range, val_loss, label='Validation Loss')
            plt.legend(loc='upper right')
            plt.title('Loss')

            plt.subplot(1, 3, 2)
            plt.plot(epochs_range, dice, label='Training Dice')
            plt.plot(epochs_range, val_dice, label='Validation Dice')
            plt.legend(loc='lower right')
            plt.title('Dice Coefficient')

            plt.subplot(1, 3, 3)
            plt.plot(epochs_range, iou, label='Training CDice')
            plt.plot(epochs_range, val_iou, label='Validation CDice')
            plt.legend(loc='lower right')
            plt.title('Classify_metrics')# IoU
            plt.show()

            input_image = tf.keras.Input(shape=(self.width, self.height, 3), name="ccinput")# "image"
            predictions,classifications = model(input_image, training = True)
            inference_model = tf.keras.Model(inputs=input_image, outputs=[predictions,classifications])# cc

            for i, test in enumerate(self.test_dataset):
                img,mask_grad= test  
                prediction,classify = inference_model.predict(img)

                img = img["ccinput"][0].numpy()
                mask = mask_grad['grad5_Conv_ReLU'][0].numpy()

                prediction = prediction[0]
                prediction = tf.math.argmax(prediction, -1)
                prediction = prediction.numpy()

                fig = plt.figure(10, figsize = (20,20))
                ax1 = fig.add_subplot(1, 3, 1)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                ax1.imshow(img)
                ax1.set_title('Image')
                ax1.axis("off")

                ax2 = fig.add_subplot(1, 3, 2)
                ax2.imshow(mask)
                ax2.set_title('Ground Truth Mask')
                ax2.axis("off")

                ax3 = fig.add_subplot(1, 3, 3)
                ax3.imshow(prediction)
                ax3.set_title('Prediction')
                ax3.axis("off")
                plt.show()

                if i == 1:
                    break
            del model
        print("Training End\n\n")
        self.optimal_k = DiceIoU_list.index(max(DiceIoU_list)) + 1
        print(f"K-Fold Cross Validation Result\nmDice : {mean_Dice*20:.3f}, mIoU : {mean_IoU*20:.3f}, Optimal_K : {self.optimal_k}\n\n")

        
    def Evaluation(self, num_sample):
        input_image = tf.keras.Input(shape=(self.width, self.height, 3), name="ccinput")# "image"
        model = Unet()
        model.load_weights(
            f"{self.model_dir}U-Net_5.h5")# {self.optimal_k}

#         model.compile(loss = self.loss_fn, 
#                       optimizer = self.optimizer,
#                       metrics = [Dice, Jaccard]
#                       )
        model.compile(loss = {'grad5_Conv_ReLU':self.loss_fn,'grad':self.loss_fc} ,#'categorical_crossentropy'   
                                  loss_weights = [1,0.8],
                                  optimizer = self.optimizer,
                                  metrics = {'grad5_Conv_ReLU':Dice,"grad":Dice_class}# Jaccard
        #                           metrics = ["accuracy"]
                                  )
        dice = model.evaluate(self.test_dataset, batch_size = self.batch_size, verbose = 1)#  _, dice, iou

        predictions,classifications = model(input_image, training=True)
        inference_model = tf.keras.Model(inputs=input_image, outputs=[predictions,classifications])
        
#         print(predictions.shape)# (None, 1024, 1024, 3)
        
        
      # cc  
        print("Display predictions")
        for i, test in enumerate(self.test_dataset):
            img, mask_grad= test
            prediction,classify = inference_model.predict(img)
            
            
            
            img = img["ccinput"][0].numpy()
            mask = mask_grad["grad5_Conv_ReLU"][0].numpy()
            
#             mask1=to_categorical(mask,3)
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            

            prediction = prediction[0]
           
            prediction = tf.math.argmax(prediction, -1)
            prediction = prediction.numpy()       
            
            fig = plt.figure(i, figsize = (20,20))
            ax1 = fig.add_subplot(1, 3, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax1.imshow(img)
            ax1.set_title('Image')
            ax1.axis("off")

            ax2 = fig.add_subplot(1, 3, 2)
            ax2.imshow(mask)
            ax2.set_title('Ground Truth Mask')
            ax2.axis("off")

            ax3 = fig.add_subplot(1, 3, 3)
            ax3.imshow(prediction)
            ax3.set_title('Prediction')
            ax3.axis("off")
            plt.show()
            
#             ax4 = fig.add_subplot(1, 4, 3)
#             ax4.plot(fpr_1, tpr_1, 'r', label='AUC = %0.2f' % auc1)

#             ax4.legend(loc='lower right')
#             ax4.plot([0, 1], [0, 1], 'r--')
#             # plt.xlim([0, 1])  # the range of x-axis
#             # plt.ylim([0, 1])  # the range of y-axis
#             ax4.xlabel('False Positive Rate')  # the name of x-axis
#             ax4.ylabel('True Positive Rate')  # the name of y-axis
#             ax4.title('Receiver operating characteristic example')  # the title of figure
            
            
            plt.show()   

#             print(hist_info(3,prediction,mask))
            
            if i == num_sample:
                break

In [ ]:
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))# sess.run(......)


vgg_CFL = MODEL()
vgg_CFL.Run_training()
sess.run(vgg_CFL.Evaluation(num_sample = -1))
del vgg_CFL

In [ ]:
saved_model.summary()

Model: "model_57"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
ccinput (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
model_56 (Functional)           [(None, 32, 32, 512) 14714688    ccinput[0][0]                    
__________________________________________________________________________________________________
ccblock0_Conv0 (Conv2D)         (None, 32, 32, 512)  2359808     model_56[0][0]                   
__________________________________________________________________________________________________
ccblock0_BN0 (BatchNormalizatio (None, 32, 32, 512)  2048        ccblock0_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock0_ReLU0 (ReLU)           (None, 32, 32, 512)  0           ccblock0_BN0[0][0]               
__________________________________________________________________________________________________
ccblock0_Conv1 (Conv2D)         (None, 32, 32, 512)  2359808     ccblock0_ReLU0[0][0]             
__________________________________________________________________________________________________
ccblock0_BN1 (BatchNormalizatio (None, 32, 32, 512)  2048        ccblock0_Conv1[0][0]             
__________________________________________________________________________________________________
ccblock0_ReLU1 (ReLU)           (None, 32, 32, 512)  0           ccblock0_BN1[0][0]               
__________________________________________________________________________________________________
ccblock1_UpSampling0 (UpSamplin (None, 64, 64, 512)  0           ccblock0_ReLU1[0][0]             
__________________________________________________________________________________________________
ccblock1_Conv0 (Conv2D)         (None, 64, 64, 512)  1049088     ccblock1_UpSampling0[0][0]       
__________________________________________________________________________________________________
ccblock1_BN0 (BatchNormalizatio (None, 64, 64, 512)  2048        ccblock1_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock1_ReLU0 (ReLU)           (None, 64, 64, 512)  0           ccblock1_BN0[0][0]               
__________________________________________________________________________________________________
ccblock1_concat (Concatenate)   (None, 64, 64, 1024) 0           ccblock1_ReLU0[0][0]             
                                                                 model_56[0][1]                   
__________________________________________________________________________________________________
ccblock1_Conv2 (Conv2D)         (None, 64, 64, 256)  2359552     ccblock1_concat[0][0]            
__________________________________________________________________________________________________
ccblock1_BN2 (BatchNormalizatio (None, 64, 64, 256)  1024        ccblock1_Conv2[0][0]             
__________________________________________________________________________________________________
ccblock1_ReLU2 (ReLU)           (None, 64, 64, 256)  0           ccblock1_BN2[0][0]               
__________________________________________________________________________________________________
ccblock2_UpSampling0 (UpSamplin (None, 128, 128, 256 0           ccblock1_ReLU2[0][0]             
__________________________________________________________________________________________________
ccblock2_Conv0 (Conv2D)         (None, 128, 128, 256 262400      ccblock2_UpSampling0[0][0]       
__________________________________________________________________________________________________
ccblock2_BN0 (BatchNormalizatio (None, 128, 128, 256 1024        ccblock2_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock2_ReLU0 (ReLU)           (None, 128, 128, 256 0           ccblock2_BN0[0][0]               
__________________________________________________________________________________________________
ccblock2_concat (Concatenate)   (None, 128, 128, 768 0           ccblock2_ReLU0[0][0]             
                                                                 model_56[0][2]                   
__________________________________________________________________________________________________
ccblock2_Conv2 (Conv2D)         (None, 128, 128, 128 884864      ccblock2_concat[0][0]            
__________________________________________________________________________________________________
ccblock2_BN2 (BatchNormalizatio (None, 128, 128, 128 512         ccblock2_Conv2[0][0]             
__________________________________________________________________________________________________
ccblock2_ReLU2 (ReLU)           (None, 128, 128, 128 0           ccblock2_BN2[0][0]               
__________________________________________________________________________________________________
ccblock3_UpSampling0 (UpSamplin (None, 256, 256, 128 0           ccblock2_ReLU2[0][0]             
__________________________________________________________________________________________________
ccblock3_Conv0 (Conv2D)         (None, 256, 256, 128 65664       ccblock3_UpSampling0[0][0]       
__________________________________________________________________________________________________
ccblock3_BN0 (BatchNormalizatio (None, 256, 256, 128 512         ccblock3_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock3_ReLU0 (ReLU)           (None, 256, 256, 128 0           ccblock3_BN0[0][0]               
__________________________________________________________________________________________________
ccblock3_concat (Concatenate)   (None, 256, 256, 384 0           ccblock3_ReLU0[0][0]             
                                                                 model_56[0][3]                   
__________________________________________________________________________________________________
ccblock3_Conv2 (Conv2D)         (None, 256, 256, 64) 221248      ccblock3_concat[0][0]            
__________________________________________________________________________________________________
ccblock3_BN2 (BatchNormalizatio (None, 256, 256, 64) 256         ccblock3_Conv2[0][0]             
__________________________________________________________________________________________________
ccblock3_ReLU2 (ReLU)           (None, 256, 256, 64) 0           ccblock3_BN2[0][0]               
__________________________________________________________________________________________________
ccblock4_UpSampling0 (UpSamplin (None, 512, 512, 64) 0           ccblock3_ReLU2[0][0]             
__________________________________________________________________________________________________
ccblock4_Conv0 (Conv2D)         (None, 512, 512, 64) 16448       ccblock4_UpSampling0[0][0]       
__________________________________________________________________________________________________
ccblock4_BN0 (BatchNormalizatio (None, 512, 512, 64) 256         ccblock4_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock4_ReLU0 (ReLU)           (None, 512, 512, 64) 0           ccblock4_BN0[0][0]               
__________________________________________________________________________________________________
ccblock4_concat (Concatenate)   (None, 512, 512, 192 0           ccblock4_ReLU0[0][0]             
                                                                 model_56[0][4]                   
__________________________________________________________________________________________________
ccblock4_Conv2 (Conv2D)         (None, 512, 512, 32) 55328       ccblock4_concat[0][0]            
__________________________________________________________________________________________________
ccblock4_BN2 (BatchNormalizatio (None, 512, 512, 32) 128         ccblock4_Conv2[0][0]             
__________________________________________________________________________________________________
ccblock4_ReLU2 (ReLU)           (None, 512, 512, 32) 0           ccblock4_BN2[0][0]               
__________________________________________________________________________________________________
ccblock5_UpSampling0 (UpSamplin (None, 1024, 1024, 3 0           ccblock4_ReLU2[0][0]             
__________________________________________________________________________________________________
ccblock5_Conv0 (Conv2D)         (None, 1024, 1024, 3 4128        ccblock5_UpSampling0[0][0]       
__________________________________________________________________________________________________
ccblock5_BN0 (BatchNormalizatio (None, 1024, 1024, 3 128         ccblock5_Conv0[0][0]             
__________________________________________________________________________________________________
ccblock5_ReLU0 (ReLU)           (None, 1024, 1024, 3 0           ccblock5_BN0[0][0]               
__________________________________________________________________________________________________
ccblock5_concat (Concatenate)   (None, 1024, 1024, 9 0           ccblock5_ReLU0[0][0]             
                                                                 model_56[0][5]                   
__________________________________________________________________________________________________
max_pooling2d_37 (MaxPooling2D) (None, 1, 1, 512)    0           model_56[0][0]                   
__________________________________________________________________________________________________
ccblock5_Conv2 (Conv2D)         (None, 1024, 1024, 1 13840       ccblock5_concat[0][0]            
__________________________________________________________________________________________________
ccflatten (Flatten)             (None, 512)          0           max_pooling2d_37[0][0]           
__________________________________________________________________________________________________
ccblock5_BN2 (BatchNormalizatio (None, 1024, 1024, 1 64          ccblock5_Conv2[0][0]             
__________________________________________________________________________________________________
dense_85 (Dense)                (None, 32)           16416       ccflatten[0][0]                  
__________________________________________________________________________________________________
ccblock5_ReLU2 (ReLU)           (None, 1024, 1024, 1 0           ccblock5_BN2[0][0]               
__________________________________________________________________________________________________
re_lu_61 (ReLU)                 (None, 32)           0           dense_85[0][0]                   
__________________________________________________________________________________________________
grad5_Conv_ReLU (Conv2D)        (None, 1024, 1024, 3 51          ccblock5_ReLU2[0][0]             
__________________________________________________________________________________________________
grad (Dense)                    (None, 5)            165         re_lu_61[0][0]                   
==================================================================================================
Total params: 24,393,544
Trainable params: 24,388,520
Non-trainable params: 5,024
__________________________________________________________________________________________________